In [1]:
from model import LLM
from pdf_parser import process_pdf

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

from langchain.chains import create_history_aware_retriever
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory

True


In [2]:
llm = LLM.get_llm(LLM.get_llm_by_id(2))
retriever = process_pdf("/Users/chaitanyabasava/Desktop/Sai Naga Viswa Chaitanya_Basava_resume.pdf")

In [3]:
contextualize_q_system_prompt = """
Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is.
"""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# takes in an input and chat_history to be invoked and returns a list of Documents
# llm is used to generate a search prompt using the input and chat_history
# this is then sent to the retriever to get a list of Documents
history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)

In [4]:
human_prompt = """
QUESTION: {input}
CONTEXT: {context}
"""

system_prompt = """
You are an assistant for question-answering tasks. Provide a clear, direct, and concise answer to the user's question, using the provided context. Your response should be no longer than three sentences, ensuring brevity and accuracy. If you don't know the answer, say that you don't know.
"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", human_prompt),
    ]
)

# takes in context (list of documents)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt, document_variable_name="context")

# first param can be a retriever or Runnable (another chain/llm) - should return a list of Documents
# all the inputs passed when invoking the chain will be passed to the retriever if it's a Runnable
# second param is a Runnable which gets the original input, the chat history, and a context with retrieved docs
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [5]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [9]:
res = conversational_rag_chain.invoke(
    {"input": "which skills should I highlight for data engineering jobs?"},
    config={
        "configurable": {"session_id": "abc123"}
    }
)

print(res["answer"])

/Users/chaitanyabasava/miniconda3/envs/revamper/lib/python3.10/site-packages/langchain_community/chat_message_histories/sql.py:186: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. Use Use connection instead instead.
  warn_deprecated(


For data engineering jobs, you should highlight your proficiency in programming languages, big data & cloud technologies, databases, and technical competencies. Specifically, your experience with Apache Spark, Hadoop, AWS, Azure, and machine learning is highly relevant and sought after in the data engineering field.
